In [146]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
# Import API key
from api_keys import g_key

In [147]:
# Load csv
path = os.path.join('..', 'weatherpy', 'weatherdata.csv' ) 
df = pd.read_csv(path)
df.head()

,Unnamed: 0,Cities,Country,Temp (F),Latitude,Longitude,Cloudiness (%),Humidity (%),Wind Speed (mph),Date
0,0,sechura,PE,67.33,-5.5569,-80.8222,62,76,10.94,1627050364
1,1,port alfred,ZA,55.81,-33.5906,26.8910,55,58,11.14,1627050365
2,2,ushuaia,AR,38.86,-54.8000,-68.3000,75,81,14.97,1627050366
3,3,barrow,US,35.62,71.2906,-156.7887,90,93,16.11,1627050367
4,4,bubaque,GW,80.55,11.2833,-15.8333,98,80,14.05,1627050076


In [148]:
# Get rid of duplicate index column
df = df.drop(columns='Unnamed: 0')


In [149]:
# configure gmaps
gmaps.configure(api_key=g_key)

coords = df[(['Latitude', 'Longitude'])]


In [150]:
# Customize the size of the figure
figure_layout = {
    'width': '700px',
    'height': '400px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

# Add the layer to the map
fig.add_layer(gmaps.heatmap_layer(coords,weights=df['Humidity (%)'], max_intensity = df['Humidity (%)'].max()))

fig

Figure(layout=FigureLayout(border='1px solid black', height='400px', margin='0 auto 0 auto', padding='1px', wi…

In [151]:
# Drop any rows that contain missing values
df_ideal = df.dropna()
# Limit max temp values, cloudiness values, and wind speed values for ideal vacation spots
df_ideal = df_ideal.loc[(df_ideal['Temp (F)'] <= 82.0) & (df_ideal['Temp (F)'] > 77.0)
                       & (df_ideal['Cloudiness (%)'] < 15) & (df_ideal['Wind Speed (mph)'] < 10.0) & 
                       (df_ideal['Wind Speed (mph)'] > 5.0)]
df_ideal


,Cities,Country,Temp (F),Latitude,Longitude,Cloudiness (%),Humidity (%),Wind Speed (mph),Date
92,stevens point,US,79.66,44.5236,-89.5746,1,81,5.75,1627050304
167,lincoln,US,81.28,40.8000,-96.6670,1,69,5.99,1627050337
222,douglas,US,81.14,41.3000,-96.1670,1,74,7.00,1627050531
258,mayo,US,79.43,38.8876,-76.5119,1,64,8.01,1627050558
331,ardmore,US,81.39,34.1743,-97.1436,1,78,5.99,1627050618
333,mporokoso,ZM,80.02,-9.3727,30.1250,3,22,7.90,1627050620
362,ascheberg,DE,81.43,51.7833,7.6167,2,48,5.50,1627050592
384,bestobe,KZ,80.06,52.4994,73.0975,0,33,7.81,1627050659
482,ondjiva,AO,77.04,-17.0667,15.7333,0,10,6.91,1627050734


In [152]:
hotel_df = df_ideal.copy()
hotel_df['Hotel Name'] = ""

# Loop through ideal cities
for index,row in hotel_df.iterrows():
    city = row['Cities']
    # get coordinates in needed format
    lat1 = (hotel_df.loc[(hotel_df['Cities'] == city)]['Latitude']).astype(str)
    lon1 = (hotel_df.loc[(hotel_df['Cities'] == city)]['Longitude']).astype(str)
    loc = lat1 + ", " + lon1
    # set up a parameters dictionary
    params = {
    'location': loc,
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
   # "rankby": "distance"
    }
    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    
    # convert response to json
    places_data = response.json()
    try:
        hotel_df.loc[index,'Hotel Name'] = places_data["results"][0]["name"]
    except:
        print(f"Results for {city} not found.")
        

#Print the json (pretty printed) for last city to get structure idea
#print(json.dumps(places_data, indent=4, sort_keys=True))
hotel_df   


Results for bestobe not found.


,Cities,Country,Temp (F),Latitude,Longitude,Cloudiness (%),Humidity (%),Wind Speed (mph),Date,Hotel Name
92,stevens point,US,79.66,44.5236,-89.5746,1,81,5.75,1627050304,Fairfield Inn & Suites by Marriott Stevens Point
167,lincoln,US,81.28,40.8000,-96.6670,1,69,5.99,1627050337,Courtyard by Marriott Lincoln Downtown/Haymarket
222,douglas,US,81.14,41.3000,-96.1670,1,74,7.00,1627050531,Hilton Garden Inn Omaha West
258,mayo,US,79.43,38.8876,-76.5119,1,64,8.01,1627050558,Resorts Resource Group International
331,ardmore,US,81.39,34.1743,-97.1436,1,78,5.99,1627050618,"Motel 6 Ardmore, OK"
333,mporokoso,ZM,80.02,-9.3727,30.1250,3,22,7.90,1627050620,Government Guest House
362,ascheberg,DE,81.43,51.7833,7.6167,2,48,5.50,1627050592,Hotel Jagdschlösschen
384,bestobe,KZ,80.06,52.4994,73.0975,0,33,7.81,1627050659,
482,ondjiva,AO,77.04,-17.0667,15.7333,0,10,6.91,1627050734,SOS Motel


In [153]:
# get coordinates as list of tuples


# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{Cities}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]
    
fig.add_layer(gmaps.marker_layer(locations, info_box_content = hotel_info))
fig

Figure(layout=FigureLayout(border='1px solid black', height='400px', margin='0 auto 0 auto', padding='1px', wi…